<a href="https://colab.research.google.com/github/datarae/algorithmic_trading/blob/main/black_scholes_and_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Options Trading Module
Author: Rae Chipera - Jaxorik AI Research Group

This file contains a Python module for options traders including a Black Scholes options
pricing model and a calculator of the Greeks, including Delta, Gamma, Theta, Vega, Rho and
Vanna. (Most models do not include Vanna, but we find it useful, so we included it.)

This code expects to be run in Google Colab with Drive mounting for complete functionality,
but the core calculation functions could be used in any Python environment.
"""

'\nOptions Trading Module\nAuthor: Rae Chipera - Jaxorik AI Research Group\n\nThis file contains a Python module for options traders including a Black Scholes options \npricing model and a calculator of the Greeks, including Delta, Gamma, Theta, Vega, Rho and \nVanna. (Most models do not include Vanna, but we find it useful, so we included it.)\n\nThis code expects to be run in Google Colab with Drive mounting for complete functionality,\nbut the core calculation functions could be used in any Python environment.\n'

In [3]:
"""
To run this module:

from trading_tools.options import BlackScholes, Greeks

Author: Rae Chipera - Jaxorik AI Research Group

"""

' \nTo run this module: \n\nfrom trading_tools.options import BlackScholes, Greeks\n\nAuthor: Rae Chipera - Jaxorik AI Research Group\n\n'

In [4]:
"""
Requirements:

numpy>=1.20.0
pandas>=1.3.0
scipy>=1.7.0
matplotlib>=3.4.0
yfinance>=0.1.70
requests>=2.26.0
beautifulsoup4>=4.10.0
mpl_toolkits.mplot3d
seaborn>=0.11.2
datetime
typing
re

"""

'\nRequirements:\n\nnumpy>=1.20.0\npandas>=1.3.0\nscipy>=1.7.0\nmatplotlib>=3.4.0\nyfinance>=0.1.70\nrequests>=2.26.0\nbeautifulsoup4>=4.10.0\nmpl_toolkits.mplot3d\nseaborn>=0.11.2\ndatetime\ntyping\nre\n\n'

In [ ]:
# This file sets up the Black-Scholes options pricing module
# Run this once to create the module files

# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

# Create directories and files
import os

# Create directories
base_path = '/content/drive/MyDrive/trading_tools'
options_path = os.path.join(base_path, 'options')
os.makedirs(options_path, exist_ok=True)

# Write files
with open(os.path.join(options_path, '__init__.py'), 'w') as f:
  f.write('from .black_scholes import BlackScholes, Greeks\n')

# Write the black_scholes.py file with all class definitions
black_scholes_content = """
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Union, Dict, List, Tuple, Optional

class BlackScholes:
    \"\"\"
    Black-Scholes option pricing model implementation.

    This class provides methods to calculate option prices and implied volatility
    using the Black-Scholes model for European options.
    \"\"\"

    @staticmethod
    def d1(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the d1 component of the Black-Scholes formula.

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The d1 component
        \"\"\"
        if sigma <= 0 or T <= 0:
            return float('nan')
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    @staticmethod
    def d2(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the d2 component of the Black-Scholes formula.

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The d2 component
        \"\"\"
        if sigma <= 0 or T <= 0:
            return float('nan')
        return BlackScholes.d1(S, K, r, sigma, T) - sigma * np.sqrt(T)

    @staticmethod
    def call_price(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the price of a European call option.

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The call option price
        \"\"\"
        if sigma <= 0 or T <= 0:
            return max(0, S - K * np.exp(-r * T))

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    @staticmethod
    def put_price(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the price of a European put option.

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The put option price
        \"\"\"
        if sigma <= 0 or T <= 0:
            return max(0, K * np.exp(-r * T) - S)

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    @staticmethod
    def implied_volatility(option_price: float, S: float, K: float, r: float, T: float,
                          option_type: str = 'call', precision: float = 0.00001,
                          max_iterations: int = 100) -> float:
        \"\"\"
        Calculate implied volatility using the Newton-Raphson method.

        Args:
            option_price (float): Market price of the option
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            T (float): Time to expiration (in years)
            option_type (str): 'call' or 'put'
            precision (float): Desired precision of the result
            max_iterations (int): Maximum number of iterations

        Returns:
            float: The implied volatility
        \"\"\"
        if option_price <= 0 or T <= 0:
            return float('nan')

        # Initial guess for volatility
        sigma = 0.2

        for i in range(max_iterations):
            if option_type.lower() == 'call':
                price = BlackScholes.call_price(S, K, r, sigma, T)
                vega = BlackScholes.vega(S, K, r, sigma, T)
            else:
                price = BlackScholes.put_price(S, K, r, sigma, T)
                vega = BlackScholes.vega(S, K, r, sigma, T)

            # If vega is too small, avoid division by zero
            if abs(vega) < 1e-8:
                return float('nan')

            diff = option_price - price
            if abs(diff) < precision:
                return sigma

            # Update sigma using Newton-Raphson
            sigma = sigma + diff / vega

            # If sigma goes negative or too large, reset
            if sigma <= 0 or sigma > 5:
                sigma = 0.2

        return float('nan')  # Failed to converge

    @staticmethod
    def vega(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Vega of an option (sensitivity to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Vega of the option (same for calls and puts)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        return S * np.sqrt(T) * norm.pdf(d1) / 100  # Divided by 100 for 1% change

    @staticmethod
    def calculate_years_to_expiry(expiry_date: str) -> float:
        \"\"\"
        Calculate the time to expiration in years.

        Args:
            expiry_date (str): Expiration date in format 'YYYY-MM-DD'

        Returns:
            float: Time to expiration in years
        \"\"\"
        expiry = datetime.strptime(expiry_date, '%Y-%m-%d')
        now = datetime.now()
        days_to_expiry = (expiry - now).days
        return max(0, days_to_expiry / 365.0)

    @staticmethod
    def option_chain_pricing(S: float, chain_df: pd.DataFrame, r: float, sigma: float = None,
                            use_market_price: bool = False) -> pd.DataFrame:
        \"\"\"
        Calculate option prices for an entire option chain.

        Args:
            S (float): Current stock price
            chain_df (pd.DataFrame): DataFrame with columns: 'strike', 'expiry',
                                    optionally 'call_price', 'put_price', 'impl_vol'
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float, optional): Volatility to use (if None, use implied vol)
            use_market_price (bool): Whether to use market prices in calculation

        Returns:
            pd.DataFrame: DataFrame with option prices and Greeks
        \"\"\"
        results = chain_df.copy()

        # Calculate time to expiry if it doesn't exist
        if 'T' not in results.columns:
            if 'expiry' in results.columns:
                results['T'] = results['expiry'].apply(BlackScholes.calculate_years_to_expiry)
            else:
                raise ValueError("DataFrame must contain either 'T' or 'expiry' column")

        # Calculate option prices and Greeks for each row
        for idx, row in results.iterrows():
            K = row['strike']
            T = row['T']

            # Determine which volatility to use
            vol = sigma
            if vol is None:
                if 'impl_vol' in row and not pd.isna(row['impl_vol']):
                    vol = row['impl_vol']
                else:
                    vol = 0.3  # Default volatility if not provided

            # Calculate prices if market prices not used
            if not use_market_price:
                results.at[idx, 'call_price'] = BlackScholes.call_price(S, K, r, vol, T)
                results.at[idx, 'put_price'] = BlackScholes.put_price(S, K, r, vol, T)

            # Calculate Greeks
            greeks = Greeks.calculate_all(S, K, r, vol, T)
            for greek_name, greek_value in greeks.items():
                results.at[idx, greek_name] = greek_value

        return results

class Greeks:
    \"\"\"
    Class for calculating all option Greeks.

    This class provides methods to calculate all first, second, and third-order
    Greeks for European options under the Black-Scholes model.
    \"\"\"

    @staticmethod
    def delta(S: float, K: float, r: float, sigma: float, T: float, option_type: str = 'call') -> float:
        \"\"\"
        Calculate the Delta of an option (sensitivity to underlying price).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)
            option_type (str): 'call' or 'put'

        Returns:
            float: The Delta of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            if option_type.lower() == 'call':
                return 1.0 if S > K else 0.0
            else:
                return -1.0 if S < K else 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)

        if option_type.lower() == 'call':
            return norm.cdf(d1)
        else:
            return norm.cdf(d1) - 1

    @staticmethod
    def gamma(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Gamma of an option (second derivative of value to underlying).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Gamma of the option (same for calls and puts)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))

    @staticmethod
    def theta(S: float, K: float, r: float, sigma: float, T: float, option_type: str = 'call') -> float:
        \"\"\"
        Calculate the Theta of an option (sensitivity to time decay).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)
            option_type (str): 'call' or 'put'

        Returns:
            float: The Theta of the option (daily time decay)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        # Common term
        common = -(S * sigma * norm.pdf(d1)) / (2 * np.sqrt(T))

        if option_type.lower() == 'call':
            theta = common - r * K * np.exp(-r * T) * norm.cdf(d2)
        else:
            theta = common + r * K * np.exp(-r * T) * norm.cdf(-d2)

        # Convert to daily theta
        return theta / 365.0

    @staticmethod
    def vega(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Vega of an option (sensitivity to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Vega of the option (same for calls and puts)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        return S * np.sqrt(T) * norm.pdf(d1) / 100  # Divided by 100 for 1% change

    @staticmethod
    def rho(S: float, K: float, r: float, sigma: float, T: float, option_type: str = 'call') -> float:
        \"\"\"
        Calculate the Rho of an option (sensitivity to interest rate).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)
            option_type (str): 'call' or 'put'

        Returns:
            float: The Rho of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d2 = BlackScholes.d2(S, K, r, sigma, T)

        if option_type.lower() == 'call':
            return K * T * np.exp(-r * T) * norm.cdf(d2) / 100  # Divided by 100 for 1% change
        else:
            return -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    @staticmethod
    def charm(S: float, K: float, r: float, sigma: float, T: float, option_type: str = 'call') -> float:
        \"\"\"
        Calculate the Charm of an option (delta decay over time).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)
            option_type (str): 'call' or 'put'

        Returns:
            float: The Charm of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        # Calculate charm
        pdf_d1 = norm.pdf(d1)

        if option_type.lower() == 'call':
            charm = -pdf_d1 * ((r / (sigma * np.sqrt(T))) - ((d2 / (2 * T))))
        else:
            charm = -pdf_d1 * ((r / (sigma * np.sqrt(T))) - ((d2 / (2 * T))))
            charm = -charm  # Negative for puts

        # Convert to daily charm
        return charm / 365.0

    @staticmethod
    def vanna(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Vanna of an option (sensitivity of delta to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Vanna of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        return -norm.pdf(d1) * d2 / sigma / 100  # Divided by 100 for 1% change

    @staticmethod
    def volga(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Volga (Vomma) of an option (second derivative of value to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Volga of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        vega = Greeks.vega(S, K, r, sigma, T)
        return vega * (d1 * d2 / sigma)

    @staticmethod
    def veta(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Veta of an option (sensitivity of vega to time).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Veta of the option (daily change)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        t1 = -S * norm.pdf(d1) * np.sqrt(T)
        t2 = r * d1 / (sigma * np.sqrt(T))
        t3 = (d1 * d2) / (2 * T)

        veta = -t1 * (t2 + t3) / 100  # Divided by 100 for 1% change

        # Convert to daily veta
        return veta / 365.0

    @staticmethod
    def speed(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Speed of an option (third derivative of value to underlying).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Speed of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)

        gamma = Greeks.gamma(S, K, r, sigma, T)
        return -gamma * (d1 / (sigma * np.sqrt(T)) + 1) / S

    @staticmethod
    def zomma(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Zomma of an option (sensitivity of gamma to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Zomma of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        gamma = Greeks.gamma(S, K, r, sigma, T)
        return gamma * ((d1 * d2 - 1) / sigma) / 100  # Divided by 100 for 1% change

    @staticmethod
    def color(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Color of an option (sensitivity of gamma to time).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Color of the option (daily change)
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        gamma = Greeks.gamma(S, K, r, sigma, T)
        t1 = r * d1 / (sigma * np.sqrt(T))
        t2 = (d1 * d2 - 1) / (2 * T)

        color = -gamma * (t1 + t2)

        # Convert to daily color
        return color / 365.0

    @staticmethod
    def ultima(S: float, K: float, r: float, sigma: float, T: float) -> float:
        \"\"\"
        Calculate the Ultima of an option (sensitivity of volga to volatility).

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            float: The Ultima of the option
        \"\"\"
        if sigma <= 0 or T <= 0:
            return 0.0

        d1 = BlackScholes.d1(S, K, r, sigma, T)
        d2 = BlackScholes.d2(S, K, r, sigma, T)

        vega = Greeks.vega(S, K, r, sigma, T)
        term1 = d1 * d2
        term2 = d1 * d1
        term3 = d2 * d2

        return (-vega / (sigma * sigma)) * (term1 * (1 - term1) + term2 * (1 - term1) - term3 * term1)

    @staticmethod
    def calculate_all(S: float, K: float, r: float, sigma: float, T: float) -> Dict[str, float]:
        \"\"\"
        Calculate all Greeks for both call and put options.

        Args:
            S (float): Current stock price
            K (float): Option strike price
            r (float): Risk-free interest rate (annual, expressed as decimal)
            sigma (float): Volatility (annual, expressed as decimal)
            T (float): Time to expiration (in years)

        Returns:
            Dict[str, float]: Dictionary with all Greek values
        \"\"\"
        greeks = {}

        # First-order Greeks
        greeks['delta_call'] = Greeks.delta(S, K, r, sigma, T, 'call')
        greeks['delta_put'] = Greeks.delta(S, K, r, sigma, T, 'put')
        greeks['theta_call'] = Greeks.theta(S, K, r, sigma, T, 'call')
        greeks['theta_put'] = Greeks.theta(S, K, r, sigma, T, 'put')
        greeks['vega'] = Greeks.vega(S, K, r, sigma, T)
        greeks['rho_call'] = Greeks.rho(S, K, r, sigma, T, 'call')
        greeks['rho_put'] = Greeks.rho(S, K, r, sigma, T, 'put')

        # Second-order Greeks
        greeks['gamma'] = Greeks.gamma(S, K, r, sigma, T)
        greeks['vanna'] = Greeks.vanna(S, K, r, sigma, T)
        greeks['charm_call'] = Greeks.charm(S, K, r, sigma, T, 'call')
        greeks['charm_put'] = Greeks.charm(S, K, r, sigma, T, 'put')
        greeks['volga'] = Greeks.volga(S, K, r, sigma, T)
        greeks['veta'] = Greeks.veta(S, K, r, sigma, T)

        # Third-order Greeks
        greeks['speed'] = Greeks.speed(S, K, r, sigma, T)
        greeks['zomma'] = Greeks.zomma(S, K, r, sigma, T)
        greeks['color'] = Greeks.color(S, K, r, sigma, T)
        greeks['ultima'] = Greeks.ultima(S, K, r, sigma, T)

        return greeks
"""

with open(os.path.join(options_path, 'black_scholes.py'), 'w') as f:
    f.write(black_scholes_content)

# Install required packages
!pip install -q numpy pandas scipy matplotlib

# Add to Python path
import sys
sys.path.append('/content/drive/MyDrive')

print("✅ Black-Scholes module setup complete!")
print(f"📁 Files created at: {options_path}")

# Verify that it works
try:
    from trading_tools.options import BlackScholes, Greeks
    print("✅ Import successful")

    # Simple example
    S = 100    # Stock price
    K = 100    # Strike price
    r = 0.05   # Risk-free rate (5%)
    sigma = 0.2 # Volatility (20%)
    T = 1.0    # Time to expiration (1 year)

    call_price = BlackScholes.call_price(S, K, r, sigma, T)
    put_price = BlackScholes.put_price(S, K, r, sigma, T)

    print(f"Call price: ${call_price:.2f}")
    print(f"Put price: ${put_price:.2f}")

    # Calculate some key Greeks
    delta = Greeks.delta(S, K, r, sigma, T, 'call')
    gamma = Greeks.gamma(S, K, r, sigma, T)
    vega = Greeks.vega(S, K, r, sigma, T)
    theta = Greeks.theta(S, K, r, sigma, T, 'call')
    vanna = Greeks.vanna(S, K, r, sigma, T)

    print(f"Delta: {delta:.4f}")
    print(f"Gamma: {gamma:.4f}")
    print(f"Vega: {vega:.4f}")
    print(f"Theta: {theta:.4f}")
    print(f"Vanna: {vanna:.4f}")

except Exception as e:
    print(f"❌ Import failed: {e}")